# APIによるデータの収集と活用

## TwitterのAPI

### REST API / Streaming API

#### REST API
１ユーザあたり15分につき15回しか呼び出せない

#### Streaming API
一度リクエストを送るとサーバとの間でコネクションが確立されたままになり、<br/>
新しいデータが生まれるたびにサーバーからデータが送られてくるプッシュ型のフロー。

 Pubric streams : Twitterのユーザ全体の公開ストリーム<br/>
    filter : 特定のキーワードやユーザで絞り込んだストリーム<br/>
    sample: 公開された全ツイートのうち一部をランダムにサンプリングしたストリーム<br/>

 User Stream : ユーザのタイムラインなど、一人のユーザに関するストリーム<br/>

## Twitter APIの認証
OAuth 1.0a による認証が必要。(ConsumerKey/ConsumerSecret/AccessToken/AccessTokenSecret)

https://apps.twitter.com

(今は電話番号登録が必要？)

In [1]:
!pip install requests-oauthlib

### OAuth1Sessionクラス  
RequestsのSessionを継承していて、コンストラクタにConsumerKeyなど4つを指定してインスタンスを作成
インスタンス作成後は認証を意識することなくRequestsのSessionオブジェクトと同じように扱える


In [2]:
!cat ../sample_codes/5-2/rest_api_with_requests_oauthlib.py

import os

from requests_oauthlib import OAuth1Session

# 環境変数から認証情報を取得する。
CONSUMER_KEY = os.environ['CONSUMER_KEY']
CONSUMER_SECRET = os.environ['CONSUMER_SECRET']
ACCESS_TOKEN = os.environ['ACCESS_TOKEN']
ACCESS_TOKEN_SECRET = os.environ['ACCESS_TOKEN_SECRET']

# 認証情報を使ってOAuth1Sessionオブジェクトを得る。
twitter = OAuth1Session(CONSUMER_KEY,
                        client_secret=CONSUMER_SECRET,
                        resource_owner_key=ACCESS_TOKEN,
                        resource_owner_secret=ACCESS_TOKEN_SECRET)

# ユーザーのタイムラインを取得する。
response = twitter.get('https://api.twitter.com/1.1/statuses/home_timeline.json')

# APIのレスポンスはJSON形式の文字列なので、response.json()でパースしてlistを取得できる。
# statusはツイート（Twitter APIではStatusと呼ばれる）を表すdict。
for status in response.json():
    print('@' + status['user']['screen_name'], status['text'])  # ユーザー名とツイートを表示する。


# Twitter API アプリケーションの権限
- 読み取り専用(ReadOnly)
- 読み書き可能(Read and write)
- 読み書き可能+ダイレクトメッセージへのアクセス

In [3]:
# キー環境変数から読み込む
import os
os.environ['SCALA_HOME']

'/usr/local/src/scala'

In [4]:
# forego スクリプトの実行時に環境変数を渡す
!brew install forego

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 2 taps (caskroom/cask, homebrew/php).
==> Updated Formulae
homebrew/php/phan@0.8                    homebrew/php/phan@0.9



In [5]:
# !forego run python my_script.py
# .envという名前のファイルから環境変数を読み取って渡してくれる。

In [6]:
#!forego run python rest_api_with_requests_oauthlib.py

# TweepyによるTwitter REST APIの利用

In [7]:
!pip install tweepy

In [8]:
!cat ../sample_codes/5-2/rest_api_with_tweepy.py

import os

import tweepy  # pip install tweepy

# 環境変数から認証情報を取得する。
CONSUMER_KEY = os.environ['CONSUMER_KEY']
CONSUMER_SECRET = os.environ['CONSUMER_SECRET']
ACCESS_TOKEN = os.environ['ACCESS_TOKEN']
ACCESS_TOKEN_SECRET = os.environ['ACCESS_TOKEN_SECRET']

# 認証情報を設定する。
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)  # APIクライアントを取得する。
public_tweets = api.home_timeline()  # ユーザーのタイムラインを取得する。

for status in public_tweets:
    print('@' + status.user.screen_name, status.text)  # ユーザー名とツイートを表示する。


In [9]:
#!forego run python ../sample_codes/5-2/rest_api_with_tweepy.py

In [10]:
!cat ../sample_codes/5-2/streaming_api_with_tweepy.py

import os

import tweepy

# 環境変数から認証情報を取得する。
CONSUMER_KEY = os.environ['CONSUMER_KEY']
CONSUMER_SECRET = os.environ['CONSUMER_SECRET']
ACCESS_TOKEN = os.environ['ACCESS_TOKEN']
ACCESS_TOKEN_SECRET = os.environ['ACCESS_TOKEN_SECRET']

# 認証情報を設定する。
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)


class MyStreamListener(tweepy.StreamListener):
    """
    Streaming APIで取得したツイートを処理するためのクラス。
    """

    def on_status(self, status):
        """
        ツイートを受信したときに呼び出されるメソッド。引数はツイートを表すStatusオブジェクト。
        """
        print('@' + status.author.screen_name, status.text)

# 認証情報とStreamListenerを指定してStreamオブジェクトを取得する。
stream = tweepy.Stream(auth, MyStreamListener())
# 公開されているツイートをサンプリングしたストリームを受信する。
# キーワード引数languagesで、日本語のツイートのみに絞り込む。
stream.sample(languages=['ja'])
